In [203]:
# tpoints = [8, 16, 31, 61, 121]
# corrects1 = [0.1620, 0.1440, 0.1310, 0.1240, 0.1040]
# corrects2 = [0.1910, 0.1660, 0.1420, 0.1310, 0.1150]
# plt.scatter([np.log2(val) for val in tpoints], corrects1, label='2000/1000')
# plt.scatter([np.log2(val) for val in tpoints], corrects2, label='4000/2000')
# plt.xlabel('Log2 of time points')
# plt.ylabel('Percentage of correct')
# plt.legend()

In [1]:
# Update iterator algorithm; and making recommendations; 
import collections
import compress_pickle
import copy
import itertools
import libsbml
import numpy as np
import operator
import os
import pickle
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")

from AMAS import annotation_maker as am
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

spec_ml_df = pd.read_csv(os.path.join(ACCURACY_DIR, 'biomd_individual_species_accuracy.csv'))
biomds_spec = np.unique(spec_ml_df['model'])
print("Number of BioModels to be analyzed for SPECIES: %d" % len(biomds_spec))

reac_ml_df = pd.read_csv(os.path.join(ACCURACY_DIR, 'biomd_individual_reactions_accuracy.csv'))
biomds_reac = np.unique(reac_ml_df['model'])
print("Number of BioModels to be analyzed for REACTIONS: %d" % len(biomds_reac))

Number of BioModels to be analyzed for SPECIES: 306
Number of BioModels to be analyzed for REACTIONS: 131


In [2]:
one_fpath = os.path.join(BIOMODEL_DIR, 'BIOMD0000000190.xml')
recom = recommender.Recommender(libsbml_fpath=one_fpath)

In [6]:
BIOMD_17_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000017.xml')
recom17 = recommender.Recommender(libsbml_fpath=BIOMD_17_PATH)

one_res = recom17.recommendAnnotation(optimize=False)
one_sub_df = one_res[one_res['id']=='AcetoinIn']
two_res = recom17.recommendAnnotation(optimize=True)
two_sub_df = two_res[two_res['id']=='AcetoinIn']

In [7]:
one_sub_df

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
54,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:2430,aconitine,1.000,0,ignore
55,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:15688,acetoin,0.943,1,keep


In [8]:
two_sub_df

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:15378,hydron,0.91675,0,ignore
1,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:15688,acetoin,0.94300,1,keep


In [3]:
BIOMD_17_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000017.xml')
recom = recommender.Recommender(libsbml_fpath=BIOMD_17_PATH)

mssc = 'top'
cutoff = 0.0
optimize = 'y'
# outtype = 'table'
# pred_spec = recom.recommendSpecies(ids=None, 
#                                    min_len=0,
#                                    mssc=mssc,
#                                    cutoff=cutoff,
#                                    outtype='table')
# pred_reac = recom.recommendReactions(ids=None, 
#                                      min_len=0,
#                                      mssc=mssc,
#                                      cutoff=cutoff,
#                                      outtype='table')
# # TODO: add optimize option
# if optimize: 
#   opt_res = recom.optimizePrediction(pred_spec,
#                                      pred_reac, 
#                                      list(np.unique(pred_reac['id'])))

pred_spec = recom.getSpeciesListRecommendation(pred_ids=recom.getSpeciesIDs(),
                                               mssc=mssc,
                                               cutoff=cutoff,
                                               get_df=True)

pred_reac = recom.getReactionListRecommendation(pred_ids=recom.getReactionIDs(),
                                               mssc=mssc,
                                               cutoff=cutoff,
                                               get_df=True)
# select data frame with at least one predicted row
filt_reac = [val for val in pred_reac if val.shape[0]>0]
filt_reac_ids = [val.index.name for val in filt_reac]

spec_formulas = dict()
for one_rec in pred_spec:
  formulas = list(set([cn.REF_CHEBI2FORMULA[k] \
                       for k in one_rec['annotation'] \
                       if k in cn.REF_CHEBI2FORMULA.keys()]))
  spec_formulas[one_rec.index.name] = formulas

# if filt_reac_ids:
#   fin_spec, fin_reac = recom.optimizePrediction(pred_spec,
#                                                 pred_reac,
#                                                 filt_reac_ids)
# res_spec_df = recom.getRecomTable(element_type='species',
#                                   recommended=fin_spec)
# res_reac = fin_reac + [val for val in pred_reac if val not in filt_reac]
# res_reac_df = recom.getRecomTable(element_type='reaction',
#                                   recommended=res_reac)
# pd.concat([res_spec_df, res_reac_df], ignore_index=True)

In [4]:
# def recommendAnnotation(mssc='top',
#                         cutoff=0.0,
#                         optimize=False,
#                         outtype='table'):
#   """
#   Combine recommendSpecies and recommendReactions
#   methods; can optimize.
  
#   Parameters
#   ----------
#   mssc: str
#   cutoff: float
#   optiimize: bool
#   outtype: str
#       If 'table', returns recommendation table
#       if 'sbml', returns an updated SBML model. 
      
#   Returns
#   -------
#   pandas.DataFrame / str
#   """
#   pred_spec = recom.getSpeciesListRecommendation(pred_ids=recom.getSpeciesIDs(),
#                                                  mssc=mssc,
#                                                  cutoff=cutoff,
#                                                  get_df=True)
#   pred_reac = recom.getReactionListRecommendation(pred_ids=recom.getReactionIDs(),
#                                                   mssc=mssc,
#                                                   cutoff=cutoff,
#                                                   get_df=True)
#   if optimize:
#     res_tab = recom.optimizePrediction(pred_spec=pred_spec,
#                                        pred_reac=pred_reac)
#   else:
#     s_df = recom.getRecomTable(element_type='species',
#                                recommended=pred_spec)
#     r_df = recom.getRecomTable(element_type='reaction',
#                                recommended=pred_reac)
#     res_tab = pd.concat([s_df, r_df], ignore_index=True)
#   return res_tab

In [5]:
res = recommendAnnotation()

In [6]:
res[res['id']=='AcetoinIn']

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
54,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:2430,aconitine,1.000,0,ignore
55,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:15688,acetoin,0.943,1,keep


In [9]:
RESULT_RECOM = cn.Recommendation('R_PFK',
                                 [('RHEA:12420', 0.6), ('RHEA:13377', 0.6)],
                                 ['https://www.rhea-db.org/rhea/12420', 'https://www.rhea-db.org/rhea/13377'],
                                 ['tagatose-6-phosphate kinase activity', 'phosphoglucokinase activity'])

In [11]:
print(recom.getMarkdownFromRecommendation(rec=RESULT_RECOM,
                                                   show_url=False))

                                   R_PFK                                    
+----+--------------+---------------+--------------------------------------+
|    | annotation   |   match score | label                                |
+====+==============+===============+======================================+
|  0 | RHEA:12420   |         0.600 | tagatose-6-phosphate kinase activity |
+----+--------------+---------------+--------------------------------------+
|  1 | RHEA:13377   |         0.600 | phosphoglucokinase activity          |
+----+--------------+---------------+--------------------------------------+


In [17]:
recom17 = recommender.Recommender(libsbml_fpath=BIOMD_17_PATH)   
specs = recom17.getSpeciesIDs()
res_spec = recom17.getSpeciesListRecommendation(pred_ids=specs,
                                                get_df=True)
reacts = recom17.getReactionIDs() 
res_reac = recom17.getReactionListRecommendation(pred_ids=reacts,
                                                 get_df=True)
opt_recom = recom17.optimizePrediction(pred_spec=res_spec,
                                       pred_reac=res_reac)

In [18]:
opt_recom = recom17.optimizePrediction(pred_spec=res_spec,
                                       pred_reac=res_reac)
sub_recom = opt_recom[opt_recom['id']=='AcetoinIn']
# self.assertTrue('AcetoinIn' in np.unique(opt_recom['id']))
# self.assertTrue('CHEBI:2430' in sub_recom['annotation'])
# self.assertTrue('CHEBI:15378' in sub_recom['annotation'])

In [26]:
print('    [307 chars]n|  0 | RHEA:12420   |         0.600 | tagatos[261 chars]---+')

    [307 chars]n|  0 | RHEA:12420   |         0.600 | tagatos[261 chars]---+


In [28]:
opt_recom[opt_recom['id']=='AcetoinIn']

,file,type,id,display name,meta id,annotation,annotation label,match score,existing,UPDATE ANNOTATION
0,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:15378,hydron,0.91675,0,ignore
1,BIOMD0000000017.xml,species,AcetoinIn,,_682168,CHEBI:15688,acetoin,0.94300,1,keep


In [29]:
'CHEBI:15378' in sub_recom['annotation']

False